# Εργαστήριο 11
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Μνήμη εντολών
Στη συνέχεια του εργαστηρίου θα χρειαστούμε μια μνήμη μόνο για ανάγνωση (ROM) για να αποθηκεύουμε τις εντολές προς εκτέλεση. Στην Chisel αυτό μπορεί να γίνει με τη χρήση του `VecInit()` όπως στο επόμενο παράδειγμα:

~~~scala
val rom = VecInit(33.U,1256.U,555.U)
~~~

Ο πιο πάνω κώδικας, όταν θα μεταφερθεί η σχεδίαση σε ένα πραγματικό κύκλωμα θα δημιουργήσει ένα τμήμα υλικού που δέχεται μια είσοδο επιλογής (διεύθυνση) και παράγει την αντίστοιχη τιμή. Στο προηγούμενο παράδειγμα θα είναι rom(0)=33, rom(1)=1256 και rom(2)=555.

*Σημ.: Ανάλογα με την τεχνολογία κατασκευής, το κύκλωμα του VecInit θα υλοποιηθεί με πραγματική δομή ROM ή με απλές πύλες.* 

### Παράδειγμα χρήσης ROM
Δείτε ένα απλό module που περιέχει ένα τμήμα ROM:

In [ ]:
class ReadOnlyMemory extends Module {
  val io = IO(new Bundle {
    val addr = Input(UInt(8.W))
    val data_out = Output(UInt(16.W))
  })
  val rom = VecInit(33.U,1256.U,555.U)
    
  io.data_out := rom(io.addr)
}

Δοκιμάστε να πάρετε την έξοδο με διάφορες διευθύνσεις. Τι παρατηρείτε στις επιστρεφόμενες τιμές όταν η διεύθυνση επιλέγει θέσεις που δεν έχουν οριστεί;

In [ ]:
test(new ReadOnlyMemory()) { c => 
  c.io.addr.poke(0.U)
  println(c.io.data_out.peek())
  c.io.addr.poke(1.U)
  println(c.io.data_out.peek())
  c.io.addr.poke(2.U)
  println(c.io.data_out.peek())
  c.io.addr.poke(3.U)
  println(c.io.data_out.peek())
  c.io.addr.poke(4.U)
  println(c.io.data_out.peek())
}

## Γενικό module μνήμης εντολών
Στη συνέχεια δίνεται ένα *παραμετρικό* module `InstructionMemory` που περιέχει ένα τμήμα ROM για εντολές. Οι παράμετροι δημιουργίας είναι οι ακόλουθες:

* `addr_w`: εύρος διεύθυνσης (πόσα bits έχει η διεύθυνση εισόδου)

* `instr_w`: εύρος εντολών (πόσα bits έχουν τα δεδομένα εξόδου)



In [ ]:
class InstructionMemory(addr_w: Int, instr_w: Int) extends Module {
  val io = IO(new Bundle {
    val address = Input(UInt(addr_w.W))
    val data_out = Output(UInt(instr_w.W))
  })
  val rom = VecInit(33.U,1256.U,555.U)
    
  io.data_out := rom(io.address)
}

## Κωδικοποίηση εντολών
Έστω ότι το σύστημά σας διαθέτει εντολές σταθερού μήκους **16 bits** και αναγνωρίζει 2 κατηγορίες εντολών:
* Αριθμητικές-λογικές πράξεις
* Ανάθεση σταθεράς σε καταχωρητή

### Αριθμητικές-λογικές πράξεις
Οι αριθμητικές-λογικές πράξεις έχουν τη μορφή:

`Rdest <- Rsrc2 Funct Rsrc1`

όπου είναι:

* `Rdest`: καταχωρητής προορισμού αποτελέσματος (0...7)
* `Rsrc2`: καταχωρητής εισόδου δεδομένων 2 (0...7)
* `Rsrc1`: καταχωρητής εισόδου δεδομένων 1 (0...7)
* `Funct`: bits επιλογής πράξης

Τα bits επιλογής πράξης έχουν την ακόλουθη μορφή:

| `Funct` | `Πράξη` |
| - | - |
| `000` | `AND` |
| `001` | `OR` |
| `010` | `XOR` |
| `011` | `+` |
| `111` | `-` |


Η κωδικοποίηση των εντολών αυτών έχει ως εξής:

| `bits 15-12` | `bits 11-9` | `bits 8-6` | `bits 5-3` | `bits 2-0` |
| --- | --- | --- | --- | --- |
| `0000` | `Rdest` | `Funct` | `Rsrc2` | `Rsrc1` |

### Ανάθεση σταθεράς σε καταχωρητή
H μορφή αυτής της εντολής είναι η ακόλουθη:

`Rdest <- Const`

όπου:

* `Rdest`: καταχωρητής προορισμού αποτελέσματος (0...7)
* `Const`: σταθερά 8 bits

Η κωδικοποίηση της εντολής αυτής έχει ως εξής:

| `bits 15-12` | `bits 11-9` | `bit 8` | `bits 7-0` |
| --- | --- | --- | --- |
| `0001` | `Rdest` | 0 | `Const` |

## Άσκηση 1
Χρησιμοποιώντας το module `InstructionMemory` που έχει δοθεί προηγουμένως, κατασκευάστε μνήμη προγράμματος που περιέχει την επόμενη ακολουθία εντολών:

`R1 <- 0`

`R2 <- 1`

`R3 <- 1`

`R1 <- R1 + R2`

`R2 <- R2 + R3`

`R1 <- R1 + R2`

`R2 <- R2 + R3`

`R1 <- R1 + R2`

`R2 <- R2 + R3`

`R1 <- R1 + R2`

`R2 <- R2 + R3`

Τι ακριβώς υπολογίζει η προηγούμενη ακολουθία εντολών;

In [ ]:
class InstructionMemory(addr_w: Int, instr_w: Int) extends Module {
  val io = IO(new Bundle {
    val address = Input(UInt(addr_w.W))
    val data_out = Output(UInt(instr_w.W))
  })
  val rom = VecInit(
                    // ...συμπληρώστε...
                   )
    
  io.data_out := rom(io.address)
}

## Άσκηση 2
Θα χρειαστείτε το module `PcLogic` που έχετε κατασκευάσει σε προηγούμενο εργαστήριο. Αντιγράψτε το στο επόμενο κελί.



In [ ]:
class PcLogic(addr_w: Int) extends Module {
  // ...συμπληρώστε...
}

Στη συνέχεια συμπληρώστε το module `FetchUnit` στο επόμενο κελί:

1. Δημιουργήστε ένα instance του module `InstructionMemory` με όνομα `imem`, με εύρος διεύθυνσης 8 bits και εύρος εντολών 16 bits:

~~~scala
val imem = Module(new InstructionMemory(8,16))
~~~
  
2. Δημιουργήστε ένα instance του module `PcLogic` με όνομα `pc_logic` με εύρος διεύθυνσης 8 bits:

~~~scala
val pc_logic = Module(new PcLogic(8))
~~~

3. Συνδέστε την είσοδο `address` του `imem` με την έξοδο `address` του `pc_logic`.


4. Το module `FetchUnit` θα έχει 2 εισόδους, `offset` και `pc_sel`, οι οποίες θα συνδεθούν στις αντίστοιχες εισόδους του `pc_logic`.


5. Το module `FetchUnit` θα έχει 1 έξοδο `instr_bits` στην οποία θα συνδεθεί η έξοδος `data_out` του `imem`.

In [ ]:
class FetchUnit extends Module {
  val io = IO(new Bundle {
    val offset = // ...συμπληρώστε...
    val pc_sel = // ...συμπληρώστε...
    val instr_bits = // ...συμπληρώστε...
  })
  
  // ...συμπληρώστε...  
    
}

Ελέγξτε την ορθότητα του module `FetchUnit`, δίνοντας παλμούς ρολογιού και παρατηρώντας αν στην έξοδο `instr_bits` εμφανίζονται τα bits που έχετε αποθηκεύσει στη μνήμη εντολών. **Η είσοδος `pc_sel` πρέπει να είναι 0 για της ανάγκες της τρέχουσας άσκησης**.

In [ ]:
test(new FetchUnit()) { c =>
  c.io.pc_sel.poke(0.U)
  println(c.io.instr_bits.peek())     // print instruction at addr 0  
  for (i <- 0 to 9) {
    c.clock.step()
    println(c.io.instr_bits.peek())   // print instructions at next addresses
  }
}